In [ ]:
IMAGE_SIZE = 224
NUM_CHANELLS = 3
LEN_FEATURES = IMAGE_SIZE * IMAGE_SIZE * NUM_CHANELLS
BATCH_SIZE = 64
# ZISE_OF_IMAGES = 4500
ZISE_OF_IMAGES = 3000
CLASSES = ['happy', 'sad']
# DRIVE_PATH = '/content/drive'
PATH = r"D:\\Projects\Development\\courses_projects\\deep_learning_and_natural_language_processing\\dog-image-mood-classification"
PATH_TO_DATA = PATH + '\\' + "data\\images"
RES_PATH = r"results\\"

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# tools
import os
import glob
import random
import numpy as np
from datetime import datetime
import pytz
import sklearn as sk
from tqdm import tqdm

# modeling
import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
tf.disable_v2_behavior()
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

In [ ]:

def split_train_test(test_size: float, images, labels, files):
  size_of_train = round(len(images) * test_size)
  train_images = images[0: size_of_train]
  train_labels = labels[0: size_of_train]
  train_files = files[0: size_of_train]

  test_images = images[size_of_train: -1]
  test_labels = labels[size_of_train: -1]
  test_files = files[size_of_train: -1]

  return [train_images, train_labels, train_files], [test_images, test_labels, test_files]

def load_data(classes):
  images = []
  labels = []
  files = []
  for i in range(len(classes)):
    folder_name = classes[i]
    path = os.path.join(PATH_TO_DATA, folder_name)
    count = 0
    for file_index, file_name in enumerate(os.listdir(path)):
      images.append(file_name)
      label = [i]
      labels.append(label)
      files.append(os.path.join(path, file_name))
      if count == ZISE_OF_IMAGES:
        break
      count += 1

  return images, labels, files

def generate_batch(file_names, labels, batch_size):
  num_of_batch = int(len(file_names)/batch_size)
  i = 0
  while(True):
    if i == num_of_batch:
      i = 0
    start = i * batch_size
    end = (i + 1) * batch_size
    images = []
    for file_name in file_names[start:end]:
      image = cv2.imread(file_name)
      image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
      images.append(image)
    yield np.array(images), np.array(labels[start:end])
    i += 1

def normalise_data(data):
  data = data.astype(np.float32)
  for i in range(len(data)):
    for j in range(len(data[i])):
      data[i][j]/=255.0
  return data

def plot_images(images, cls_true, cls_pred=None):
    
    if len(images) == 0:
        print("no images to show")
        return 
    else:
        random_indices = random.sample(range(len(images)), min(len(images), 9))
        
    if cls_pred is not None:
        images, cls_true, cls_pred  = zip(*[(images[i], cls_true[i], cls_pred[i]) for i in random_indices])
    else:
        images, cls_true  = zip(*[(images[i], cls_true[i]) for i in random_indices])
    
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i].reshape(IMAGE_SIZE, IMAGE_SIZE, 3))

        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        ax.set_xlabel(xlabel)
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

In [ ]:
images, labels, files = load_data(CLASSES)
images, labels, files = shuffle(images, labels, files)  # shuffle the data

In [ ]:
train, test = split_train_test(test_size=0.8, images=images, labels=labels, files=files)
train_images, train_labels, train_files = train[:]
test_images, test_labels, test_files = test[:]

In [ ]:
print(f"Size of train: {len(train_images)}")
print(f"Size of test: {len(test_images)}")

In [ ]:
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

# tf Graph Input
x = tf.placeholder(tf.float32, [None, 22500]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 1]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.zeros([22500, 10]))
b = tf.Variable(tf.zeros([1]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))